# Research Goal

> The reason I requested this data is to begin testing the SoundThinking claim that the majority of gunfire events in a city go unreported by humans.
>
> My goal is to compare ShotSpotter and 911 calls about gunfire by block to see if they are 1-1.

# FOIA request
### Ref. No: **F249914-110424**
[Link to FOIA portal](https://chicagoil.govqa.us/WEBAPP/_rs/(S(bxbpx2fwoohqvcdmh3nl31bl))/IssueSearch.aspx?sSessionID=)
- You're supposed to be able to search by Ref. No without signing in, but I tried a couple times and it kept taking me to a sign-in page.
- After a couple minutes it processed the search, but I tried several variations of the Ref. No and none returned any records.

### Contents of request & response
>Dear Bailey Passmore:
>
>On behalf of the Office of Emergency Management and Communications (OEMC), I am responding to your Freedom of Information (FOIA) request that our office received on November 04, 2024, wherein you requested the following information:
>
>"This is a request for public records under the Illinois Freedom of Information Act. Please make available for my inspection the following public records in the department’s possession, custody or control.
>
>I would like a copy of the database including all records of 911 calls, requests for service, and emergency events dispatched through OEMC. Specifically, I am interested in records between January 1, 2018 to present and involving a report of gunfire, such as "SHOTSPOTTER", "SHOTS FIRED", "PERSON SHOT", etc.
>
>Please include the following fields:
>- call or event ID field
>- date of call or event
>- date of dispatch
>- date cleared
>- date closed
>- location of reported emergency (at least a block level address)
>- district of reported emergency
>- priority code assigned
>- initial event type
>- final event type
>
>If possible, I would like the records to be made available as an Excel file. If the data must be separated into separate files or sheets due to the number of records, that's fine."
>
>In response to your request, the OEMC is providing you with the redacted 9-1-1 report. Be advised, residential addresses have been redacted from the attached records pursuant to 7(1)(b) of FOIA, which provides that public bodies may redact information that is considered private.  5 ILCS 2(c-5) of FOIA defines private information as follows:
>
>"Private information" means unique identifiers, including a person's social security number, driver's license number, employee identification number, biometric identifiers, personal financial information, passwords or other access codes, medical records, home or personal telephone numbers and email addresses.  Private information also includes home address and personal license plates, except as otherwise provided by law or when compiled without possibility of attribution to any person.
>
>Personal information contained within public records, the disclosure of which would constitute a clearly unwarranted invasion of personal privacy, unless the disclosure is consented to in writing by the individual subjects of the information.
>"Unwarranted invasion of personal privacy" means the disclosure of information that is highly personal or objectionable to a reasonable person and in which the subject's right to privacy outweighs any legitimate public interest in obtaining the information.
>
>To the extent a portion of your FOIA request has been denied, you have a right of review by the Illinois Attorney General's Public Access Counselor who can be contacted at 500 S. Second St., Springfield, IL 62706 or by telephone at (877)-299-FOIA.
>
>Email: public.access@ilag.gov.
>
>You may also seek judicial review of a denial under 5 ILCS 140/11 of FOIA.
>
>Sincerely,
>
>M. Mason - OEMC FOIA
>Freedom of Information Officer
>Office of Emergency Management & Communications
>1411 W Madison
>Chicago IL 60607
>312-746-9403

# Exploring the responsive records
### Initial Notes

- [x] years 2018 and 2019 are not included, and 2020 records only cover November, December
- [x] There is no appearance of "SHOT SPOTTER" (how it appears in the old data) in the new data

### Actions taken
- [x] I responded to the email notifying me about the responsive records and asked about:
    - [x] absence of 2018, 2019 records (noticed partial 2020 records later)
    - [x] how to identify ShotSpotter events in the data
- [x] Then I merged the new data with the older OEMC data and noticed that the `shotspotter` indicator is False for 100% of them. It appears these records have been intentionally excluded from the responsive records, although my request clearly specifies their inclusion.
    - [ ] I have not followed up with CPD/OEMC yet to highlight this absence, I'll wait a couple days for their response about the excluded years and classification of ShotSpotter. Additionally, the absence of most of 2020 needs to be addressed.

### Initial Findings
- The responsive records fail to address the FOIA request in two ways:
    1. Records are missing for years 2018, 2019, and most of 2020
        - "I am interested in records between January 1, 2018 to present" in request
    3. Records are missing for gunfire reports initiated by ShotSpotter technology.
        - "I am interested in records... involving a report of gunfire, such as “SHOTSPOTTER”, “SHOTS FIRED”, “PERSON SHOT”, etc."

# Processing new OEMC data

Below is a sample copied from the OEMC_MP data processing pipeline that is used in a script to validate the data before processing begins.

```
call_date:
    description: 'The date and time of the 911 call or the date an analyst relayed a suspected crime identified by technology such as Shot Spotter or HunchLab.'
    sample value: 16-SEP-2021 01:05
    percent unique: 16.8% (2,039,809 of 12,159,582)
    percent with data: 100.0% (12,159,582 of 12,159,582)
```

# Setup

In [1]:
# dependencies
import re
import pandas as pd

In [2]:
# support methods
def format_datetime(x):
    if pd.isna(x): return None
    out = pd.to_datetime(x, format="%m/%d/%Y %H:%M:%S")
    return out

In [3]:
# main
raw = pd.read_csv("../frozen/cpd_foia249914-fixed.csv", header=0)
raw.EventNumber = raw.EventNumber.astype(str)
rules = {
    'EventNumber': 'event_no',
    'EntryDate': 'entry_date',
    'DispatchDate': 'disp_date',
    'ClearDate': 'clear_date',
    'ClosedDate': 'close_date',
    'InitialPriority': 'init_priority',
    'FinPriority': 'fin_priority',
    'InitType': 'init_type',
    'FinalType': 'fin_type',
    'FinTypeDescription': 'fin_type_desc',
    'District': 'district_nov24',
    'Location': 'location_ofcall', # not sure about that
    'ServiceLocation': 'location',
    'XCoord': 'location_x',
    'YCoord': 'location_y',
}
oemc_gunfire = raw.rename(columns=rules)

datecols = [col for col in oemc_gunfire.columns if 'date' in col.lower()]
for col in datecols:
    oemc_gunfire[col] = oemc_gunfire[col].apply(lambda x: x.replace('.', ":") if pd.notna(x) else None)
    oemc_gunfire[col] = oemc_gunfire[col].apply(format_datetime)

In [4]:
oemc_gunfire[datecols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77340 entries, 0 to 77339
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   entry_date  77340 non-null  datetime64[ns]
 1   disp_date   76850 non-null  datetime64[ns]
 2   clear_date  76829 non-null  datetime64[ns]
 3   close_date  77340 non-null  datetime64[ns]
dtypes: datetime64[ns](4)
memory usage: 2.4 MB


# Preview of new data

In [5]:
oemc_gunfire.shape[0]

77340

In [6]:
oemc_gunfire.sample().T

,40117
event_no,2225315684
entry_date,2022-09-10 23:00:21
disp_date,2022-09-10 23:01:08
clear_date,2022-09-10 23:45:10
close_date,2022-09-10 23:45:10
location_ofcall,58XX N FAIRFIELD AV
location,58XX N FAIRFIELD AV
location_x,41.987
location_y,-87.698
district_nov24,20.0


In [7]:
print("# OEMC data from recent request for gunfire incidents (November 2024):")
print(f"- {oemc_gunfire.shape[0]:,} records total.")
print(f"- {(oemc_gunfire.init_type == 'SHOT SPOTTER').sum():,} records originating as a SHOTSPOTTER report.")
print(f"- Records cover a period of \
{round((oemc_gunfire.entry_date.max() - oemc_gunfire.entry_date.min()).days / 365, 1)} years \
between {oemc_gunfire.entry_date.min().date()} and {oemc_gunfire.entry_date.max().date()}.")

# OEMC data from recent request for gunfire incidents (November 2024):
- 77,340 records total.
- 0 records originating as a SHOTSPOTTER report.
- Records cover a period of 4.0 years between 2020-11-01 and 2024-11-05.


# Looking at timeline of new data
- I asked for January 1, 2018 to present

In [8]:
oemc_gunfire['year_entered'] = oemc_gunfire.entry_date.dt.year

In [9]:
oemc_gunfire.year_entered.value_counts().sort_index()

year_entered
2020     3073
2021    22684
2022    19960
2023    16401
2024    15222
Name: count, dtype: int64


**Notes from BP:**
- 2018 and 2019 are not in the summary as expected

# Looking for 'SHOT SPOTTER' in new data

In [10]:
(oemc_gunfire.init_type == 'SHOT SPOTTER').sum()

np.int64(0)

In [11]:
(oemc_gunfire.fin_type == 'SHOT SPOTTER').sum()

np.int64(0)

In [12]:
(oemc_gunfire.fin_type_desc == 'SHOT SPOTTER').sum()

np.int64(0)

In [13]:
(oemc_gunfire.init_type.str.contains('spot', flags=re.I)).sum()

np.int64(0)

In [14]:
(oemc_gunfire.fin_type.str.contains('spot', flags=re.I)).sum()

np.int64(0)

In [15]:
(oemc_gunfire.fin_type_desc.str.contains('spot', flags=re.I)).sum()

np.int64(0)

In [16]:
oemc_gunfire[['init_type', 'fin_type', 'fin_type_desc']].value_counts()

init_type  fin_type  fin_type_desc
SHOTSF     SHOTSF    SHOTS FIRED      68134
PERSHO     PERSHO    PERSON SHOT       8288
PERGUN     SHOTSF    SHOTS FIRED        196
SHOTSF     PERSHO    PERSON SHOT        162
PERSHO     SHOTSF    SHOTS FIRED         58
                                      ...  
MARINE     SHOTSF    SHOTS FIRED          1
NFY        PERSHO    PERSON SHOT          1
OEM+B      SHOTSF    SHOTS FIRED          1
OVERDO     PERSHO    PERSON SHOT          1
VICEIP     SHOTSF    SHOTS FIRED          1
Name: count, Length: 104, dtype: int64

In [17]:
oemc_gunfire.init_type.value_counts()

init_type
SHOTSF    68297
PERSHO     8346
PERGUN      226
CHECWB       42
EMS          31
          ...  
CSAJO         1
BATRT         1
EXPLOS        1
CALL          1
PERWTD        1
Name: count, Length: 75, dtype: int64

In [18]:
oemc_gunfire.fin_type.value_counts()

fin_type
SHOTSF    68704
PERSHO     8628
SHOTS         8
Name: count, dtype: int64

In [19]:
oemc_gunfire.fin_type_desc.value_counts()

fin_type_desc
SHOTS FIRED         68704
PERSON SHOT          8628
SHOTS FIRED (OV)        8
Name: count, dtype: int64


**Notes from BP:**
- I'm not seeing any records for ShotSpotter as expected

# Recovering "SHOT SPOTTER" using records in both old and new data

- This new OEMC data appears to classify both 911 calls to report gunfire and ShotSpotter events in the same way. (At least as far as the Initial and Final event types are concerned, which is where this keyphrase appears in the OEMC_MP data)

So, let's take the earlier data and connect it to this data.

### OEMC data from MP investigation

In [20]:
full_oemc_mp = pd.read_parquet("../../../OEMC_MP/export/output/oemc_dispatch.parquet")
keep = [
    'event_no',
    'district',
    'call_date',
    'disp_date',
    'on_date',
    'clear_date',
    'close_date',
    'init_priority',
    'init_type',
    'fin_type',
    'dispatch_reported',
    'numeric_district',
    'day_called',
    'year_called',
    'year_dispatched',
    'time_to_arrive',
    'tta_group',
    'shotspotter',
    'gunshot',
]
oemc_mp = full_oemc_mp[keep].copy()

In [21]:
print("# OEMC data from missing persons investigation (December 2021):")
print(f"- {full_oemc_mp.shape[0]:,} records total.")
print(f"- {full_oemc_mp.shotspotter.sum():,} records originating as a SHOTSPOTTER report.")
print(f"- Records cover a period of \
{round((oemc_mp.call_date.max() - oemc_mp.call_date.min()).days / 365, 1)} years \
between {full_oemc_mp.call_date.min().date()} and {full_oemc_mp.call_date.max().date()}.")

# OEMC data from missing persons investigation (December 2021):
- 12,159,582 records total.
- 95,242 records originating as a SHOTSPOTTER report.
- Records cover a period of 4.0 years between 2018-01-01 and 2021-12-31.


### OEMC data in both old and new sets

In [22]:
both = pd.merge(oemc_gunfire, oemc_mp,
                on='event_no',
                suffixes=('_nov24', '_dec21'),
                how='inner')

In [23]:
print("# OEMC data in both dec21 and nov24 datasets:")
print(f"- {both.shape[0]:,} records total.")
print(f"- {both.shotspotter.sum():,} records originating as a SHOTSPOTTER report.")
print(f"- Records cover a period of \
{round((both.call_date.max() - both.call_date.min()).days / 365, 1)} years \
between {both.call_date.min().date()} and {both.call_date.max().date()}.")

# OEMC data in both dec21 and nov24 datasets:
- 25,604 records total.
- 0 records originating as a SHOTSPOTTER report.
- Records cover a period of 1.2 years between 2020-11-01 and 2021-12-31.


In [24]:
both.sample().T

,22999
event_no,2131709984
entry_date,2021-11-13 15:55:59
disp_date_nov24,2021-11-13 15:57:04
clear_date_nov24,2021-11-13 16:43:11
close_date_nov24,2021-11-13 16:43:11
location_ofcall,48XX W BELMONT AV /32XX N CICERO AV
location,48XX W BELMONT AV /32XX N CICERO AV
location_x,41.938
location_y,-87.746
district_nov24,16.0


In [25]:
assert not both.shotspotter.isna().any()
both.shotspotter.value_counts()

shotspotter
False    25604
Name: count, dtype: int64

In [26]:
both[['init_type_dec21', 'fin_type_dec21', 'init_type_nov24', 'fin_type_nov24',]].value_counts()

init_type_dec21            fin_type_dec21  init_type_nov24  fin_type_nov24
SHOTS FIRED                SHOTS FIRED     SHOTSF           SHOTSF            22691
PERSON SHOT                PERSON SHOT     PERSHO           PERSHO             2642
SHOTS FIRED                PERSON SHOT     SHOTSF           PERSHO               53
PERSON WITH A GUN          SHOTS FIRED     PERGUN           SHOTSF               47
PERSON SHOT                SHOTS FIRED     PERSHO           SHOTSF               20
                                                                              ...  
INFO. FOR THE POLICE       PERSON SHOT     INFOPO           PERSHO                1
INJURED PERSON REPORT      PERSON SHOT     INJRT            PERSHO                1
MARINE DISTRESS            SHOTS FIRED     MARINE           SHOTSF                1
MENTAL HEALTH DISTURBANCE  PERSON SHOT     DISTME           PERSHO                1
THEFT IP                   PERSON SHOT     THEFTI           PERSHO                1
N


**Notes from BP:**
- It looks like these records will not be salvageable for my research goal.
- A new dataset is needed, at least containing the Event No. and a suitable indicator for ShotSpotter. That (and complete records for 2018, 2019, 2020) could be merged with these records and then I should be able to address my question.